In [3]:
from pyspark.sql import HiveContext
import pandas as pd
import json

In [4]:
import requests

In [5]:
url = "https://api.cartolafc.globo.com/partidas/1"

In [6]:
response = requests.get(url)

with open('test_cartola_outputfile.json', 'wb') as outf:
    outf.write(response.content)



In [7]:
data = json.load(open('test_cartola_outputfile.json'))

print(teste)

NameError: name 'teste' is not defined

In [8]:
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


In [9]:
sqlContext = HiveContext(sc)

In [10]:
sqlContext.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [11]:
print(type(data))

<class 'dict'>


In [12]:
#df = sqlContext.read.json(json.dumps(data))

df = spark.read.json(sc.parallelize([data]))
df.schema

StructType(List(StructField(_corrupt_record,StringType,true)))

In [13]:
spark

## Colocando os arquivos da pasta cartola no HDFS:
### hadoop fs -put /cartola/* /cartola

In [14]:
lances_2014_df = spark.read.csv("/cartola/data/2014/2014_lances.csv", header=True)
sorted_lances_2014_df = lances_2014_df.sort(lances_2014_df.ID.desc())

times_2014_df = spark.read.csv("/cartola/data/2014/2014_times.csv", header=True)
sorted_times_2014_df = times_2014_df.sort(times_2014_df.ID.desc())


In [15]:
sorted_lances_2014_df.schema
sorted_times_2014_df.schema

StructType(List(StructField(ID,StringType,true),StructField(Nome,StringType,true),StructField(Abreviacao,StringType,true),StructField(Slug,StringType,true)))

In [12]:
sorted_lances_2014_df.show()

print(sorted_lances_2014_df.count())

+---------+---------+-------+--------+-------+-------+----+
|       ID|PartidaID|ClubeID|AtletaID|Periodo|Momento|Tipo|
+---------+---------+-------+--------+-------+-------+----+
|141622764|   180149|    275|   38435|    2tr|     38|  ZB|
|141622763|   180149|    275|   87420|    2tr|     24|  ZF|
|141622762|   180149|    275|   41519|    2tr|     24|  ZB|
|141622761|   180149|    275|   88368|    1tr|     18|  ZB|
|141622760|   180149|    293|   87009|    1tr|     10|  ZB|
|141622759|   180152|    266|   50417|    2tr|     45|  CA|
|141622758|   180152|    266|   37627|    2tr|     38|  CA|
|141622757|   180152|    266|   74059|    2tr|     17|  CA|
|141622756|   180152|    266|   80370|    1tr|     40|  CA|
|141622755|   180152|    283|   42116|    1tr|     16|  CA|
|141622754|   180152|    283|   50358|    2tr|     14|  ZG|
|141622753|   180152|    283|   38573|    1tr|     44|  ZG|
|141622752|   180152|    266|   38162|    1tr|     33|  ZG|
|141622751|   180158|    284|   78715|  

In [13]:
sorted_times_2014_df.show()

print(sorted_times_2014_df.count())

+---+-------------+----------+-------------+
| ID|         Nome|Abreviacao|         Slug|
+---+-------------+----------+-------------+
|316|  figueirense|       FIG|  figueirense|
|315|  chapecoense|       CHA|  chapecoense|
|294|     coritiba|       CFC|     coritiba|
|293|  atlético-pr|       CAP|  atletico-pr|
|292|        sport|       SPO|        sport|
|290|        goiás|       GOI|        goias|
|288|     criciúma|       CRI|     criciuma|
|287|      vitória|       VIT|      vitoria|
|285|internacional|       INT|internacional|
|284|       grêmio|       GRE|       gremio|
|283|     cruzeiro|       CRU|     cruzeiro|
|282|  atlético-mg|       CAM|  atletico-mg|
|277|       santos|       SAN|       santos|
|276|    são paulo|       SAO|    sao-paulo|
|275|    palmeiras|       PAL|    palmeiras|
|266|   fluminense|       FLU|   fluminense|
|265|        bahia|       BAH|        bahia|
|264|  corinthians|       COR|  corinthians|
|263|     botafogo|       BOT|     botafogo|
|262|     

### Lendo todas as partidas

In [ ]:
tds_partidas_df = spark.read.csv("/cartola/data/*/*_partidas.csv", header=True)

tds_partidas_df.schema

In [ ]:
desc_sorted_partidas_df = tds_partidas_df.sort(tds_partidas_df._c0.desc())

desc_sorted_partidas_df.show(10)

desc_sorted_partidas_df.count()

In [ ]:
asc_sorted_partidas_df = tds_partidas_df.sort(tds_partidas_df._c0.asc())

asc_sorted_partidas_df.show(10)

asc_sorted_partidas_df.count()

### Convert DataFrame Row Object to a new DataFrame

Convert spark DataFrame column of type list to new data frame

In [ ]:
df.schema

In [ ]:
atletas_test = df.select("atletas").rdd.flatMap(lambda x: x[0]).collect()


atletas_df = spark.createDataFrame(atletas_test) 
atletas_df.head(3)

# Onde o novo dataframe foi salvo?
# atletas_df.write.json("teste/atletas_teste.json")
atletas_df.schema

In [ ]:
atletas_df.createOrReplaceTempView("tempAtletas") 


# /user/hive/warehouse/teste_tabela_atletas/

In [ ]:
sqlContext.sql("create table atletas_hive_test as select * from tempAtletas");


In [ ]:
sqlContext.sql("show databases").show()

In [ ]:
sqlContext.sql("show tables from default").show()

In [ ]:
CREATE EXTERNAL TABLE atletas (apelido STRING, atleta_id BIGINT)
  STORED AS PARQUET LOCATION '/user/hive/warehouse/teste_tabela_atletas/';

In [ ]:
# sqlContext.sql("show databases").show()

sqlContext.sql("CREATE EXTERNAL TABLE atletas (apelido STRING, atleta_id BIGINT) \
               STORED AS PARQUET LOCATION '/user/root/teste/atletas_teste.parquet' ").show()

In [ ]:
sc.setProperty("hive.metastore.uris", "thrift://hive-metastore:9083");

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
spark.sql("show databases").show()

In [ ]:
import pyspark
conf = pyspark.SparkConf()

In [ ]:
conf.set("spark.executor.uri", "thrift://hive-metastore:9083")

In [ ]:
#spark = pyspark.SparkContext(conf=conf)

In [ ]:
spark.sql("show databases").show()

In [ ]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option("driver", "com.facebook.presto.jdbc.PrestoDriver") \
    .option("url", "jdbc:presto://presto:8080/hive/default") \
    .option("user", "hive") \
    .option("dbtable", "pessoas") \
    .load()